**Libraries**

In [19]:
import requests
import sys
import math
import pandas as pd
import time
import os
import json
import numpy as np
import datetime as dt
import tzdata
from zoneinfo import ZoneInfo
import ast
import reprlib
import pprint
import sempy.fabric as fabric
import collections as coll
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize
from operator import itemgetter
from importlib.metadata import version
# from pyspark.sql.types import *
# from delta.tables import *
# import janitor as jn
# import delta.tables as dt
from typing import List, Union, Optional

# import pyspark.sql.functions as F
# from pyspark.sql import DataFrame, Column
import re

from deltalake import write_deltalake



# def add_missing_columns(df1: DataFrame, df2: DataFrame) -> DataFrame:
#     """
#     Adds to first dataframe columns from the second dataframe that don't exist in first one.
#     Columns get null values casted to respective data types
#     :param df1: first dataframe
#     :param df2: second dataframe
#     :return: new dataframe with added columns
#     """
#     additional_cols = [F.lit(None).cast(field.dataType).alias(field.name)
#                        for field in df2.schema.fields if field.name not in df1.columns]
#     return df1.select("*", *additional_cols)
print(version('pandas'))
# 2024-10-08
pd.options.mode.copy_on_write = True

# 2025-02-05
pd.options.display.max_rows = 75
pd.options.display.min_rows = 50

# import warnings
# warnings.filterwarnings('once')
# warnings.simplefilter('once')

2.1.4


In [20]:
api_key = ""

destination_lakehouse_abfss_path = ""
storage_options = {"bearer_token": notebookutils.credentials.getToken('storage'), "use_fabric_endpoint": "true"}

In [21]:
variables = ['SERIALNO','SPORDER','AGEP','ESR', 'HHT', 'HHT2', 'RELSHIPP', 'NP', 'NPF', 'HINCP','FINCP', 'PINCP','ADJINC','POVPIP','WGTP','PWGTP', 'PUMA','FS','SFN', 'DIS', 'SSIP', 'KIT', 'TYPEHUGQ']

# state = 6
state = '06'

# PUMAs - pk
# pumas_sf = ['0607507', '0607508', '0607509', '0607510', '0607511', '0607512', '0607513', '0607514']
# pumas_marin = ['0604103', '0604104']
pumas_sf = ['07507', '07508', '07509', '07510', '07511', '07512', '07513', '07514']
pumas_marin = ['04103', '04104']
pumas = pumas_sf + pumas_marin

# base_url = "https://api.census.gov/data/2023/acs/acs5/pums?get={}&for=state:{}&key={}"
base_url = "https://api.census.gov/data/2023/acs/acs5/pums?get={}&for=public%20use%20microdata%20area:{}&in=state:{}&key={}"

# request_url = base_url.format(",".join(variables),state, api_key)
request_url = base_url.format(",".join(variables), ",".join(pumas), state, api_key)

#get requests
r = requests.get(request_url)

# Check if the request was successful
if r.status_code == 200:

    # Load the JSON response
    census = json.loads(r.text)

    # Extract headers and data rows
    headers = census[0]
    data = census[1:]

    # Create DataFrame with headers
    df = pd.DataFrame(data, columns=headers)

    # Display the DataFrame
    print(df.head())
else:
    print("Failed to fetch data from the API: status code ", str(r.status_code))


        SERIALNO SPORDER AGEP ESR HHT HHT2 RELSHIPP NP NPF   HINCP  ... PWGTP  \
0  2019GQ0000080       1   31   6   0    0       37  1   1  -60000  ...     8   
1  2019GQ0000338       1   24   6   0    0       38  1   1  -60000  ...     3   
2  2019GQ0000878       1   77   6   0    0       37  1   1  -60000  ...    15   
3  2019GQ0000939       1   89   6   0    0       37  1   1  -60000  ...     3   
4  2019GQ0001782       1   18   1   0    0       38  1   1  -60000  ...    24   

    PUMA FS SFN DIS  SSIP KIT TYPEHUGQ state public use microdata area  
0  04104  2   0   2     0   0        2    06                     04104  
1  07509  2   0   2     0   0        3    06                     07509  
2  04104  2   0   1     0   0        2    06                     04104  
3  07512  2   0   1  7300   0        2    06                     07512  
4  07508  2   0   2     0   0        3    06                     07508  

[5 rows x 25 columns]


In [22]:
print(os.linesep.join(r.text.split(os.linesep)[:50]))

[["SERIALNO","SPORDER","AGEP","ESR","HHT","HHT2","RELSHIPP","NP","NPF","HINCP","FINCP","PINCP","ADJINC","POVPIP","WGTP","PWGTP","PUMA","FS","SFN","DIS","SSIP","KIT","TYPEHUGQ","state","public use microdata area"],
["2019GQ0000080","1","31","6","0","0","37","1","1","-60000","-60000","0","1.207712","-1","0","8","04104","2","0","2","0","0","2","06","04104"],
["2019GQ0000338","1","24","6","0","0","38","1","1","-60000","-60000","830","1.207712","-1","0","3","07509","2","0","2","0","0","3","06","07509"],
["2019GQ0000878","1","77","6","0","0","37","1","1","-60000","-60000","6000","1.207712","-1","0","15","04104","2","0","1","0","0","2","06","04104"],
["2019GQ0000939","1","89","6","0","0","37","1","1","-60000","-60000","7300","1.207712","-1","0","3","07512","2","0","1","7300","0","2","06","07512"],
["2019GQ0001782","1","18","1","0","0","38","1","1","-60000","-60000","7500","1.207712","-1","0","24","07508","2","0","2","0","0","3","06","07508"],
["2019GQ0002044","1","39","6","0","0","38","1","1"

In [23]:
print(len(df))
display(df)
df_orig = df.copy()

51387


In [24]:
display(df[25000:26000])

In [25]:
# display(df)
print(df.dtypes)
convert_dict = {
    'SERIALNO': str, 
    'SPORDER': 'Int32',
    'AGEP': 'Int32',
    'ESR':  'Int32',
    'HHT':  'Int32',
    'HHT2':  'Int32',
    'RELSHIPP':  'Int32',
    'NP':  'Int32',
    'NPF':  'Int32',
    'HINCP':  'Int32',
    'FINCP':  'Int32',
    'PINCP':  'Int32',
    'ADJINC': 'Float64',
    'POVPIP':  'Int32',
    'WGTP':  'Int32',
    'PWGTP':  'Int32',
    'PUMA': str,
    'FS': 'Int32',
    'SFN': 'Int32',
    'DIS': 'Int32',
    'SSIP':  'Int32',
    'KIT': 'Int32',
    'TYPEHUGQ':  'Int32',
    'state': str,
    'public use microdata area': str
}
# df['AGEP'] = df['AGEP'].astype('Int32')

# df = df.convert_dtypes()
df = df.astype(convert_dict)
display(df)
print(df.dtypes)

SERIALNO                     object
SPORDER                      object
AGEP                         object
ESR                          object
HHT                          object
HHT2                         object
RELSHIPP                     object
NP                           object
NPF                          object
HINCP                        object
FINCP                        object
PINCP                        object
ADJINC                       object
POVPIP                       object
WGTP                         object
PWGTP                        object
PUMA                         object
FS                           object
SFN                          object
DIS                          object
SSIP                         object
KIT                          object
TYPEHUGQ                     object
state                        object
public use microdata area    object
dtype: object


SERIALNO                      object
SPORDER                        Int32
AGEP                           Int32
ESR                            Int32
HHT                            Int32
HHT2                           Int32
RELSHIPP                       Int32
NP                             Int32
NPF                            Int32
HINCP                          Int32
FINCP                          Int32
PINCP                          Int32
ADJINC                       Float64
POVPIP                         Int32
WGTP                           Int32
PWGTP                          Int32
PUMA                          object
FS                             Int32
SFN                            Int32
DIS                            Int32
SSIP                           Int32
KIT                            Int32
TYPEHUGQ                       Int32
state                         object
public use microdata area     object
dtype: object


In [26]:
df_hdg_hu = df.loc[((df['AGEP'] >= 60) | ((df['AGEP'] >= 18) & (df['DIS'] == 1))) & (df['POVPIP'] <= 200) & (df['POVPIP'] >= 0) & (df['TYPEHUGQ'] == 1)] # housing units, no GQ
df_hdg_gq3 = df.loc[((df['AGEP'] >= 60) | ((df['AGEP'] >= 18) & (df['DIS'] == 1))) & (df['POVPIP'] <= 200) & (df['POVPIP'] >= 0) & (df['TYPEHUGQ'] == 3)] # noninstitutional GQ

df_hdg_hu_hh = df_hdg_hu.drop_duplicates(subset='SERIALNO')
df_hu = df.loc[(df['TYPEHUGQ'] == 1)]
df_hu_hh = df_hu.drop_duplicates(subset='SERIALNO')

hdg_pumas = pd.concat([df_hdg_hu.groupby('PUMA')['PWGTP'].sum(), df_hdg_hu_hh.groupby('PUMA')['WGTP'].sum(), df_hu_hh.groupby('PUMA')['WGTP'].sum(), df_hdg_gq3.groupby('PUMA')['PWGTP'].sum()], axis=1, ignore_index=True).reset_index()

hdg_pumas = hdg_pumas.rename(columns={0: "hdg_eligible_hu_persons", 1: "hdg_eligible_hu_households", 2: "total_hu_households", 3: "hdg_eligible_gq_persons"}, errors="raise")
# display(hdg_pumas)
# print(hdg_pumas.columns)

In [28]:
# profile for geoid

variables = ['NAME', 'GEO_ID', 'PUMA']

state = '06'

# PUMAs
# pumas_sf = ['0607507', '0607508', '0607509', '0607510', '0607511', '0607512', '0607513', '0607514']
# pumas_marin = ['0604103', '0604104']
pumas_sf = ['07507', '07508', '07509', '07510', '07511', '07512', '07513', '07514']
pumas_marin = ['04103', '04104']
pumas = pumas_sf + pumas_marin

# base_url = "https://api.census.gov/data/2023/acs/acs5/pums?get={}&for=state:{}&key={}"
base_url = "https://api.census.gov/data/2023/acs/acs5/profile?get={}&for=public%20use%20microdata%20area:{}&in=state:{}&key={}"


request_url = base_url.format(",".join(variables), ",".join(pumas), state, api_key)

r2 = requests.get(request_url)

if r2.status_code == 200:

    # Load the JSON response
    census = json.loads(r2.text)

    # Extract headers and data rows
    headers = census[0]
    data = census[1:]

    # Create DataFrame with headers
    df2 = pd.DataFrame(data, columns=headers)

    # Display the DataFrame
    print(df2.head())
else:
    print("Failed to fetch data from the API: status code ", str(r2.status_code))

                                                NAME            GEO_ID   PUMA  \
0  Marin County (North & West)--Novato & San Rafa...  795P200US0604103  04103   
1  Marin County (Southeast)--San Rafael (South), ...  795P200US0604104  04104   
2  San Francisco County (South Central)--Bayview ...  795P200US0607507  07507   
3  San Francisco County (Northwest)--Richmond, We...  795P200US0607508  07508   
4  San Francisco County (Northeast)--Chinatown, N...  795P200US0607509  07509   

  state public use microdata area  
0    06                     04103  
1    06                     04104  
2    06                     07507  
3    06                     07508  
4    06                     07509  


In [30]:
hdg_pumas.columns = [re.sub('[\W]', '_', c) for c in hdg_pumas.columns]
df2.columns = [re.sub('[\W]', '_', c) for c in df2.columns]

# schema-enabled lakehouse
write_deltalake(destination_lakehouse_abfss_path + "Tables/dbo/acs_puma_hdg_eligible", data=hdg_pumas, mode='overwrite', schema_mode='merge', engine='rust', storage_options=storage_options)
write_deltalake(destination_lakehouse_abfss_path + "Tables/dbo/acs_pumas", data=df2, mode='overwrite', schema_mode='merge', engine='rust', storage_options=storage_options)

In [ ]:
# spark_hdg_pumas = spark.createDataFrame(hdg_pumas)
# print(spark_hdg_pumas.schema)
# spark_hdg_pumas = spark_hdg_pumas.toDF(*[re.sub('[^\w]', '_', c) for c in spark_hdg_pumas.columns])
# spark_hdg_pumas.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("acs_puma_hdg_eligible")